Agora vamos vários modelos de lda que irão permitir agrupar os documentos.

In [2]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import scispacy
import spacy
import en_core_sci_lg
import joblib
import os

Em primeiro lugar carregamos um modelo de nlp treinado para documentos de biomedicina. Assim poderemos remover as stop words e convert as palavras na sua forma mais simples usando lemmatization

In [ ]:
# medium model
nlp=en_core_sci_lg.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 3000000

def spacy_tokenizer(sentence):
    #return word_lemma of non necessary words
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)]

Como as palavras que aparecem com grande ou pouco frequência no dataset podem ter influencia no desempenho do modelo, criamos a função create_customize_words() que irá definir se adicionamos as palavras muito frequentes e pouco frequentes às stop_words.

In [6]:
def create_customize_words():
    #read words and frequency
    words=pd.read_csv('data/word_count.csv')

    customize_stop_words=[]
    for index, row in words.iterrows():
        if(row['Frequency']<100 or row['Frequency']>5000): customize_stop_words.append(row['Word'])

    return customize_stop_words

Também definimos uma função que irá permitir perceber o funcionamento do LDA treinado. A função print_top_words() vai apresentar na consola os tópicos textuais mais importantes para cada um dos componentes, ou seja, sub-doenças

In [4]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

Agora, de forma a criarmos e treinarmos o modelo, implementamos a função train_lda() que nos permite definir o treino e as suas carateristicas.
Tais como o dataset a usar e se vamos eliminar ou não as palavras mais frequentes

Fora estas pequenas alterações, o que a função faz em geral é:

1-Definir as stop_words

2-Vetorizar o texto

3-Inicializar o LDA com 32 componentes

4-Treinar o LDA com o texto vetorizado

5-Calcular as distancias de cada elemento do dataset a cada tópico

6-Executar a função print_top_words()

7-Guardas os objetos resultados para poderem ser usados mais tarde

In [7]:
def train_lda(df,folder_path,delete_by_frequency):
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    customize_stop_words=[]
    #New stop words list 
    if(delete_by_frequency): customize_stop_words = create_customize_words()

    # Mark them as stop words
    for w in customize_stop_words:
        if(not isinstance(w, float)): nlp.vocab[w].is_stop = True

    #Convert a collection of text documents to a matrix of token counts
    vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)

    #Learn the vocabulary dictionary and return document-term matrix.
    #The astype(‘U’) is telling numpy to convert the data to Unicode (essentially a string in python 3)
    data_vectorized = vectorizer.fit_transform(df['summary'].values.astype('U'))

    lda = LatentDirichletAllocation(n_components=32, random_state=0)
    lda.fit(data_vectorized)

    #get topic distances
    doc_topic_dist = pd.DataFrame(lda.transform(data_vectorized))

    print_top_words(lda, vectorizer, n_top_words=25)

    #joblib.dump Persist an arbitrary Python object into one file.
    joblib.dump(vectorizer, folder_path+'/vectorizer.csv')
    joblib.dump(data_vectorized,folder_path+'/data_vectorized.csv')
    joblib.dump(lda, folder_path+'/lda.csv')
    doc_topic_dist.to_csv(folder_path+'/doc_topic_dist.csv', index=False)

## Modelo 1 

Para este modelo usamos os dataset dataset_gastric_cancer.csv sem nenhuma alteração ao dataset inicial de modo a obtermos um modelo de base

In [3]:
df = pd.read_csv('data/dataset_gastric_cancer.csv', sep='#')
train_lda(df,'baseline',False)

## Modelo 2

Para este modelo usamos os dataset dataset_gastric_cancer.csv adicionando as palavras muito e pouco frequentes às stop_words

In [ ]:
train_lda(df,'not_baseline',True)

## Modelo 3
Para este modelo usamos os dataset dataset_reduced.csv. Com este dataset iremos treinar o modelo apenas com as primeiras 100 palavras de cada publicação

In [ ]:
df = pd.read_csv('data/dataset_reduced.csv', sep='#')
train_lda(df,'reduced',False)